In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [3]:
source('paths.sh')

In [4]:
PRS202110_f %>%
fread(
    colClasses = c('#FID'='character', 'IID'='character'),
    select=c('#FID', 'IID', 'population', 'split', 'age', 'age0', 'sex', 'Array')
) %>%
rename_with(
    function(x){str_replace(x, '#', '')}, starts_with("#")
) -> phe_all_df


In [12]:
pops <- c('white_british', 'non_british_white', 'african', 's_asian', 'e_asian')


In [16]:
phe_all_df %>%
filter(population %in% pops) -> phe_df


In [17]:
phe_df %>%
nrow

[1] 378066

In [20]:
phe_df %>% 
count(population, split) %>% arrange(-n)

population,split,n
<chr>,<chr>,<int>
white_british,train,235991
white_british,test,67425
white_british,val,33713
non_british_white,non_british_white,24905
s_asian,s_asian,7831
african,african,6497
e_asian,e_asian,1704


In [8]:
235991 + 33713

[1] 269704

In [9]:
235991 + 33713 + 67425

[1] 337129

In [45]:
phe_df %>%
group_by(split) %>%
summarise(
    n = n(),
    age0_mean = mean(age0),
    age0_sd = sd(age0),
    sex_M = mean(sex),
    Array_UKBB = mean(Array)
) %>%
left_join(
    phe_df %>% 
    count(population, split), 
    by = c("split", "n")
) %>%
arrange(-n) %>%
bind_rows(
    phe_df %>%
    summarise(
        n = n(),
        age0_mean = mean(age0),
        age0_sd = sd(age0),
        sex_M = mean(sex),
        Array_UKBB = mean(Array)
    ) %>%
    mutate(population = 'Total', split='N/A')
) %>%
select(
    population, split, n, age0_mean, age0_sd, sex_M, Array_UKBB
) -> pops_tbl_df


In [47]:
pops_tbl_df %>%
rename('#population' = 'population') %>%
fwrite('pop_characteristics.tsv', sep='\t', na = "NA", quote=F)
